In [4]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import iaaft
import xarray as xr
import pandas as pd

In [7]:
data = Dataset("X:/CMIP6_2.6_CESM2_regridded.nc", 'r')

# print variables 
for var in data.variables:
    print(var)

lat = data.variables['lat']
lon = data.variables['lon']
time = data.variables['time']
#realization = data.variables['realization'][:]
tp = data.variables['tas']

print(lat)
print(lon)
print(time)
#print(realization)
print(tp)
time[:366]

lat
lon
tas
time
<class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    _FillValue: nan
    units: degrees_north
unlimited dimensions: 
current shape = (37,)
filling on
<class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    _FillValue: nan
    units: degrees_east
unlimited dimensions: 
current shape = (72,)
filling on
<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    _FillValue: nan
    units: days since 0001-01-01
    calendar: 365_day
    cell_methods: time: mean
unlimited dimensions: time
current shape = (28835,)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 tas(time, lat, lon)
    _FillValue: nan
    cell_methods: time: mean
unlimited dimensions: time
current shape = (28835, 37, 72)
filling on


masked_array(data=[737665., 737666., 737667., 737668., 737669., 737670.,
                   737671., 737672., 737673., 737674., 737675., 737676.,
                   737677., 737678., 737679., 737680., 737681., 737682.,
                   737683., 737684., 737685., 737686., 737687., 737688.,
                   737689., 737690., 737691., 737692., 737693., 737694.,
                   737695., 737696., 737697., 737698., 737699., 737700.,
                   737701., 737702., 737703., 737704., 737705., 737706.,
                   737707., 737708., 737709., 737710., 737711., 737712.,
                   737713., 737714., 737715., 737716., 737717., 737718.,
                   737719., 737720., 737721., 737722., 737723., 737724.,
                   737725., 737726., 737727., 737728., 737729., 737730.,
                   737731., 737732., 737733., 737734., 737735., 737736.,
                   737737., 737738., 737739., 737740., 737741., 737742.,
                   737743., 737744., 737745., 73774

Compute Anomalies

In [10]:
def create_new_dataset(original_ds, start_date='2022-01-01', end_date='2100-12-31'):
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Assicurati che ogni anno abbia 365 giorni
    dates = dates[~((dates.month == 2) & (dates.day == 29))]
    
    new_ds = xr.Dataset(
        {
            'tas': (('time', 'lat', 'lon'), np.full((len(dates), len(original_ds['lat']), len(original_ds['lon'])), np.nan))
        },
        coords={
            'lat': original_ds['lat'],
            'lon': original_ds['lon'],
            'time': ('time', dates)
        }
    )
    
    new_ds['time'].attrs['units'] = 'days since 2022-01-01 00:00:00'
    new_ds['tas'].attrs = original_ds['tas'].attrs  # Copia gli attributi se necessario

    return new_ds

def compute_anomalies(ds, VARIABLE, BASELINE_INTERVAL):
    start, end = BASELINE_INTERVAL
    ds = ds[VARIABLE]
    ds_baseline = ds.sel(time=(ds['time.year'] >= start) & (ds['time.year'] <= end))
    
    gb = ds_baseline.groupby('time.dayofyear')
    clim = gb.mean(dim='time')
    std_clim = gb.std(dim='time')
    
    clim_time = clim.sel(dayofyear=ds.time.dt.dayofyear)
    std_clim_time = std_clim.sel(dayofyear=ds.time.dt.dayofyear)

    anomalies = (ds - clim_time)
    return anomalies

# Carica il dataset originale
original_ds = xr.open_dataset("X:/CMIP6_8.5_CESM2_regridded.nc")

# Crea un nuovo dataset con la stessa struttura del dataset originale
new_ds = create_new_dataset(original_ds)

# Copia i dati della temperatura dal dataset originale al nuovo dataset
new_ds['tas'].loc[:, :, :] = original_ds['tas'].values

# Anomalie rispetto al periodo di riferimento 2022-2041
BASELINE_INTERVAL = [2022, 2041]
anomalies_bs222041 = compute_anomalies(new_ds, 'tas', BASELINE_INTERVAL)


In [11]:
# Crea il nuovo dataset NetCDF4
Datifiltrati = nc.Dataset('C:/Users/David/OneDrive/Desktop/CLIMATE_NETWORK/data/surr_IAAFT_t2m_2022_2100_5grid_lowemission.nc', 'w', format='NETCDF4')

# Definisci le dimensioni
num_dim = Datifiltrati.createDimension('num', 51)
time_dim = Datifiltrati.createDimension('time', 28854)  # Assumendo che 'time' sia la dimensione temporale
lat_nuova = Datifiltrati.createDimension('lat', 37)
lon_nuova = Datifiltrati.createDimension('lon', 72)

# Crea le variabili
time_var = Datifiltrati.createVariable('time', np.float32, ('time',))
lat_var = Datifiltrati.createVariable('lat', np.float32, ('lat',))
lon_var = Datifiltrati.createVariable('lon', np.float32, ('lon',))
num_var = Datifiltrati.createVariable('num', np.int32, ('num',))
t2m_var = Datifiltrati.createVariable('sh_t2m', np.float32, ('num', 'time', 'lat', 'lon'))
lat_var[:] = lat[:]
lon_var[:] = lon[:]

print(lat_var)
print(lon_var)
print(Datifiltrati.variables.keys())
print(num_var)
print(t2m_var)

# Chiudi il dataset
Datifiltrati.close()
data.close()

<xarray.DataArray 'tas' (time: 28835, lat: 37, lon: 72)>
array([[[-2.71220474, -2.71220093, -2.71219788, ..., -2.68533859,
         -2.68534546, -2.68539886],
        [ 2.30345383,  2.17094345,  1.93781433, ..., -0.46655273,
          0.44419403,  2.16784592],
        [-3.59134979, -2.93646698, -2.41069946, ..., -3.38693237,
         -3.52401962, -3.53930969],
        ...,
        [-3.71315231, -4.94169464, -6.33228455, ...,  0.98127975,
          0.12587662, -1.24362259],
        [ 3.98586884,  4.59426041,  5.12243347, ...,  4.43522873,
          3.94692154,  3.78556747],
        [10.38276978, 10.38276978, 10.38276978, ..., 10.38276978,
         10.38276978, 10.38276978]],

       [[-3.87868042, -3.87868042, -3.87868118, ..., -3.86092529,
         -3.86092529, -3.86093063],
        [-1.53252487, -1.83513336, -2.11040497, ..., -2.0475853 ,
         -1.82087479, -1.24751663],
        [-3.00951462, -1.4981636 , -1.10960922, ..., -4.02302399,
         -3.17718735, -3.61441345],
...
        [13.7504097 , 12.54686508, 11.84035721, ..., 12.07288208,
         15.679245  , 15.44793091],
        [18.70410233, 18.6006958 , 18.55266571, ..., 19.58971786,
         19.22907791, 18.87695541],
        [17.31378403, 17.31378403, 17.31378403, ..., 17.31378403,
         17.31378403, 17.31378403]],

       [[ 5.44861069,  5.44861298,  5.44861298, ...,  5.46029892,
          5.46028976,  5.46026688],
        [ 4.76950607,  5.17283554,  5.68461151, ...,  4.24258118,
          4.28882904,  4.45116653],
        [ 4.44357681,  5.8602829 ,  6.57607422, ...,  4.13769684,
          3.03362732,  3.20954132],
        ...,
        [14.23916016, 13.36763992, 11.63645477, ..., 11.48741455,
         15.58646469, 15.52775879],
        [20.42669907, 20.30074615, 20.04778442, ..., 21.31487961,
         20.96607666, 20.60245743],
        [19.11086655, 19.11086655, 19.11086655, ..., 19.11086655,
         19.11086655, 19.11086655]]])
Coordinates:
  * lat        (lat) float64 -90.0 -85.0 -80.0 -75.0 ... 75.0 80.0 85.0 90.0
  * lon        (lon) float64 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0
  * time       (time) datetime64[ns] 2022-01-01 2022-01-02 ... 2100-12-31
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365

In [12]:
# New dataset NetCDF4
Datifiltrati = nc.Dataset('X:/surr_CMIP6_2.6_CESM2_regridded.nc', 'w', format='NETCDF4')

# Define dimentions
num_dim = Datifiltrati.createDimension('num', 31)
time_dim = Datifiltrati.createDimension('time', 28835)  # Assumendo che 'time' sia la dimensione temporale
lat_nuova = Datifiltrati.createDimension('lat', 37)
lon_nuova = Datifiltrati.createDimension('lon', 72)

# Creaation variabiles
time_var = Datifiltrati.createVariable('time', np.float32, ('time',))
lat_var = Datifiltrati.createVariable('lat', np.float32, ('lat',))
lon_var = Datifiltrati.createVariable('lon', np.float32, ('lon',))
num_var = Datifiltrati.createVariable('num', np.int32, ('num',))
t2m_var = Datifiltrati.createVariable('tas', np.float32, ('num', 'time', 'lat', 'lon'))
lat_var[:] = lat[:]
lon_var[:] = lon[:]

print(lat_var)
print(lon_var)
print(Datifiltrati.variables.keys())
print(num_var)
print(t2m_var)

# Close the dataset
Datifiltrati.close()

<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
unlimited dimensions: 
current shape = (37,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
unlimited dimensions: 
current shape = (72,)
filling on, default _FillValue of 9.969209968386869e+36 used
dict_keys(['time', 'lat', 'lon', 'num', 'tas'])
<class 'netCDF4._netCDF4.Variable'>
int32 num(num)
unlimited dimensions: 
current shape = (31,)
filling on, default _FillValue of -2147483647 used
<class 'netCDF4._netCDF4.Variable'>
float32 tas(num, time, lat, lon)
unlimited dimensions: 
current shape = (31, 28835, 37, 72)
filling on, default _FillValue of 9.969209968386869e+36 used


In [13]:
# DATA INPUT
data = Dataset('X:/CMIP6_2.6_CESM2_regridded.nc', 'r')
#data = Dataset('./data/shuffled_t2m_1970_2022_4grid.nc', 'r')
lat  = data.variables['lat']        
lon  = data.variables['lon']            
temp = data.variables['tas']
print(temp)


# DATA OUTPUT
Datifiltrati = Dataset('X:/surr_CMIP6_2.6_CESM2_regridded.nc', 'r+')
lat_var = Datifiltrati.variables['lat']
lon_var = Datifiltrati.variables['lon']
num_var = Datifiltrati.variables['num']
sh_t2m = Datifiltrati.variables['tas']
print(lat_var)
print(lon_var)
print(num_var,'\n\n\n')
print(sh_t2m)

# NODES
lon_range = range(0, len(lon))
lat_range = range(0, len(lat))
nodes = tuple((i,j) for i in lat_range for j in lon_range)

'''for indi,nod in enumerate(nodes):
    print(indi, '\t', nod    )'''

# PARAMETERS
num_surr = 30
years_analysed = 1      # years intervals
#tot_years = 2023-1970
tot_years = 2101-2022

periods = tot_years//years_analysed    
print(periods)

years_analysed = 1  # Ogni anno


start_year = 2022
end_year = 2100

# Calcola il numero totale di anni nel range
total_years = end_year - start_year + 1

'''# Leap Years
start_days = []

# Calcola il giorno di inizio per ogni anno tenendo conto degli anni bisestili
current_day = 0  # Partiamo dal giorno 0, cioè il 1 gennaio 2022
for year in range(start_year, end_year + 1):
    start_days.append(current_day)
    # Controlla se l'anno è bisestile
    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
        days_in_year = 366
    else:
        days_in_year = 365
    current_day += days_in_year

last_day_index = 28835
start_days.append(28835)
'''
# No Leap Years
start_days = []

# Calcola il giorno di inizio per ogni anno tenendo conto degli anni bisestili
current_day = 0  # Partiamo dal giorno 0, cioè il 1 gennaio 2022
for year in range(start_year, end_year + 1):
    start_days.append(current_day)
    days_in_year = 365  # Assumiamo che ogni anno abbia 365 giorni
    current_day += days_in_year

# Aggiungi l'ultimo giorno per coprire tutto il range temporale
last_day_index = temp.shape[0]  # Assumiamo che temp.shape[0] sia la lunghezza della serie temporale totale
start_days.append(last_day_index)

### RANDOMIZE Surrogates
'''
sh_t2m[0, :, :, :] = temp[:, :, :]

surrogate_matrix = np.empty((num_surr, temp.shape[0], len(lat_range), len(lon_range)))

# Randomizzazione vettorializzata per ciascun surrogato
for n in range(num_surr+1):
    print(n)
    for dec in range(periods):
        print(dec)
        start, end = start_days[dec], start_days[dec+1] if dec < periods - 1 else temp.shape[0]
        
        # Seleziona il blocco di dati per l'anno corrente e randomizzalo
        year_block = temp[start:end, :, :]  # Selezione dell'intero blocco per l'anno
        reshaped_block = year_block.reshape(year_block.shape[0], -1)  # Ristrutturazione per randomizzazione
        
        # Randomizzazione
        for idx in range(reshaped_block.shape[1]):
            np.random.shuffle(reshaped_block[:, idx])
        
        # Riassegnazione al surrogato corretto, mantenendo la struttura originale
        surrogate_matrix[n, start:end, :, :] = reshaped_block.reshape(year_block.shape)

# Assumi sh_t2m[0, :, :, :] già riempito con dati originali
sh_t2m[1:, :, :, :] = surrogate_matrix

scatter_plot(temp[:, 1, 1], sh_t2m[11, :, 1, 1])
'''

'''for i in lat_range:
    for j in lon_range:
        print(f'Surrogating node ({i},{j}):')
        sh_t2m[0, :, i, j] = temp[:, i, j]  # Assegna i dati originali al primo layer
        
        # Inizializza una matrice temporanea per tutti i surrogati di questo nodo
        # Dimensioni: [num_surr, lunghezza della serie temporale totale]
        surrogate_matrix = np.zeros((num_surr, len(temp[:, i, j])))

        # Cicla attraverso ogni periodo
        for dec in range(periods):
            start, end = start_days[dec], start_days[dec+1]
            serie_temporale = temp[start:end, i, j]

            # Genera tutti i surrogati per questo periodo e assegnali nella matrice temporanea
            for n in range(num_surr):
                serie_randomizzata = np.random.permutation(serie_temporale)
                surrogate_matrix[n, start:end] = serie_randomizzata

        # Assegna la matrice dei surrogati a sh_t2m per questo nodo
        for n in range(num_surr):
            sh_t2m[n+1, :, i, j] = surrogate_matrix[n, :]

        scatter_plot(sh_t2m[0, :, i, j], sh_t2m[1, :, i, j])
        scatter_plot(sh_t2m[0, :, i, j], sh_t2m[50, :, i, j])
'''
### IAAFT Surrogates
                

for i in lat_range:
    for j in lon_range:
        print(f'Surrogating node ({i},{j}) :')
        sh_t2m[0,:,i,j] = temp[:,i,j]
        # surrogates generatioom
        all_surrogates = [[] for _ in range(num_surr)]
        for dec in range(periods):
            sh_serieiaaft1 = iaaft.surrogates(x=temp[start_days[dec]:start_days[dec+1], i, j], ns=num_surr, verbose=False )
 
            for n in range(num_surr):
                for value in sh_serieiaaft1[n]:
                    all_surrogates[n].append(value)  # Assuming this is the surrogate data generated

        for n2 in range(num_surr):                                        
            sh_t2m[n2+1,:, i, j] = all_surrogates[n2]


# Chiudi il dataset
Datifiltrati.close()
data.close()

<class 'netCDF4._netCDF4.Variable'>
float32 tas(time, lat, lon)
    _FillValue: nan
    cell_methods: time: mean
unlimited dimensions: time
current shape = (28835, 37, 72)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
unlimited dimensions: 
current shape = (37,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
unlimited dimensions: 
current shape = (72,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
int32 num(num)
unlimited dimensions: 
current shape = (31,)
filling on, default _FillValue of -2147483647 used 



<class 'netCDF4._netCDF4.Variable'>
float32 tas(num, time, lat, lon)
unlimited dimensions: 
current shape = (31, 28835, 37, 72)
filling on, default _FillValue of 9.969209968386869e+36 used
79
Surrogating node (0,0) :
Surrogating node (0,1) :
Surrogating node (0,2) :
Surrogating node (0,3) :
Surrogating node (0,4) :
Surrogating node (0,5) :
Surrog

In [ ]:
import dask.array as da
from dask.distributed import LocalCluster, Client
import dask

# Avviare il client Dask
cluster = LocalCluster(n_workers=24)
client = Client(cluster)
client

In [ ]:
import math
import numpy as np
import os
from numpy.lib.stride_tricks import as_strided
import xarray as xr


def crosscorrelation(x, y, maxlag, normalize=True):
    """
    Cross correlation with a maximum number of lags, with optional normalization.

    Parameters:
    x, y: one-dimensional numpy arrays with the same length.
    maxlag: maximum lag for which the cross correlation is computed.
    normalize: if True, calculate the normalized cross-correlation.
    
    Returns:
    An array of cross-correlation values with length 2*maxlag + 1.
    
    Credits: 
    https://stackoverflow.com/questions/30677241/how-to-limit-cross-correlation-window-width-in-numpy
    
    """
    
    py = np.pad(y.conj(), 2*maxlag, mode='constant')
    T = as_strided(py[2*maxlag:], shape=(2*maxlag+1, len(y) + 2*maxlag),
                   strides=(-py.strides[0], py.strides[0]))
    px = np.pad(x, maxlag, mode='constant')

    
    cross_corr = T.dot(px)
    
    '''if normalize:
        # Subtract means
        mx = np.mean(x)
        my = np.mean(y)
        norm = float(len(y) * mx * my)

        # Compute the numerator as the sum of products of differences from means
        numerator = cross_corr - norm

        # Compute the denominator
        sigma_x = np.sqrt(np.sum((x - mx) ** 2))
        sigma_y = np.sqrt(np.sum((y - my) ** 2))
        denominator = float(sigma_x * sigma_y)
        
        # Normalize the cross-correlation
        cross_corr = numerator / denominator'''

    cmax = float(cross_corr.max())
    the_lagmax = cross_corr.argmax() - (max_lag + 1)
    
    if normalize:
        mx = np.mean(x)
        my = np.mean(y)
        norm = float(len(y) * mx * my)

        
        numerator = cmax - norm

        sigma_x = np.sqrt(np.sum((x - mx) ** 2))
        sigma_y = np.sqrt(np.sum((y - my) ** 2))
        denominator = float(sigma_x * sigma_y)
        
        cmax = numerator / denominator

    return cmax, the_lagmax


def analyze_chunk(data_chunk, index, nodes, max_lag, dec):
    numpy_array = np.zeros((37*72, 37*72))  
    if index ==0:
        timelag_array = np.zeros((37*72, 37*72))
    
    for indi, nod in enumerate(nodes):
        Ai, Aj = nodes[indi]
        for indj, node in enumerate(nodes):
            (Bi, Bj) = nodes[indj]

            if indi < indj:
                crossmax, the_lagmax = crosscorrelation(data_chunk.values[:,Ai, Aj], data_chunk.values[:,Bi, Bj], max_lag)

                # Undirected Network
                #crossmax = cross_corr.max()
                

                numpy_array[indi, indj] = crossmax
                if index == 0:
                    timelag_array[indi, indj] = the_lagmax


                '''  # Directed Network
                first_half = cross_corr[:max_lag + 1]
                max_first_half = np.max(first_half)
                second_half = cross_corr[max_lag:]
                max_second_half = np.max(second_half)
                numpy_array[indi, indj] = max_first_half                
                numpy_array[indj, indi] = max_second_half   
                
                if index ==0:
                    lag_first_half = np.argmax(first_half) - max_lag
                    lag_second_half = np.argmax(second_half)
                    timelag_array[indi, indj] = lag_first_half
                    timelag_array[indj, indi] = lag_second_half
                
                '''

    np.save(f'X:/cross correlation max 2022 2100 CMIP6_2.6_CESM2/corrs_{index}_year_{dec}.npy', numpy_array)
    if index == 0:
        np.save(f'X:/Timelag 2022 2100 CMIP6_2.6_CESM2/timelag_{index}_year_{dec}.npy', timelag_array)

if __name__ == "__main__":

    # DATA OUTPUT
    foutpath = "X:/cross correlation max 2022 2100 CMIP6_2.6_CESM2"
    if not os.path.exists(foutpath):
        os.makedirs(foutpath)
    foutpath2 = "X:/Timelag 2022 2100 CMIP6_2.6_CESM2"
    if not os.path.exists(foutpath2):
        os.makedirs(foutpath2)

    # PARAMETERS

    max_lag = 30*5           #five months
    num_surrogates = 30
    start_year = 2022
    end_year = 2100

    '''# Leap Years
    start_days = []

    # Calcola il giorno di inizio per ogni anno tenendo conto degli anni bisestili
    current_day = 0  # Partiamo dal giorno 0, cioè il 1 gennaio 2022
    for year in range(start_year, end_year + 1):
        start_days.append(current_day)
        # Controlla se l'anno è bisestile
        if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
            days_in_year = 366
        else:
            days_in_year = 365
        current_day += days_in_year

    last_day_index = 28835
    start_days.append(28835)
    '''
    # No Leap Years
    start_days = []

    # Calcola il giorno di inizio per ogni anno tenendo conto degli anni bisestili
    current_day = 0  # Partiamo dal giorno 0, cioè il 1 gennaio 2022
    for year in range(start_year, end_year + 1):
        start_days.append(current_day)
        days_in_year = 365  # Assumiamo che ogni anno abbia 365 giorni
        current_day += days_in_year

    # Aggiungi l'ultimo giorno per coprire tutto il range temporale
    last_day_index = temp.shape[0]  # Assumiamo che temp.shape[0] sia la lunghezza della serie temporale totale
    start_days.append(last_day_index)

    # Caricare il dataset NetCDF
    ds = xr.open_dataset('X:/surr_CMIP6_2.6_CESM2_regridded.nc', chunks={'num': 1})
    #print(ds)
    sh_t2m = ds['t2m']
    lon = ds['lon']
    lat = ds['lat']
    #print(lon)
    #print(lat)


    lon_range = range(0,len(lon))
    lat_range = range(0,len(lat))
    nodes = tuple((i,j) for i in lat_range for j in lon_range)


    # DASK parallelation
    
    delayed_results = []

    for i in range(num_surrogates+1):
        for dec in range(len(start_days) - 1):  # -1 perché l'ultimo elemento in start_days non ha un periodo successivo
            data_chunk = sh_t2m.isel(num=i, time=slice(start_days[dec], start_days[dec+1]))
            delayed_results.append(dask.delayed(analyze_chunk)(data_chunk, i, nodes, max_lag, dec))
    dask.visualize(*delayed_results, optimize_graph=True, filename='my_graph.svg')
    
    
    # Calcolare i risultati
    computed_results = dask.compute(*delayed_results)